In [1]:
import time
import numpy as np
import mxnet as mx
from mxnet import gluon, nd
import math
from relgraphconv import *
from utils import *
import random
from matplotlib import pyplot as plt
from model import *
import dgl
plt.style.use('science')

Using backend: mxnet


In [2]:
seed = 1130
random.seed(seed)
np.random.seed(seed)
mx.random.seed(seed)
dgl.random.seed(seed)

In [3]:
# load data
matfn = 'adjacency_matrix_13.mat'
g, tratim_matrix = load_adjmatrix(matfn)
edge_type = np.load(
    'edge_type_13.npy')
edge_type = mx.nd.array(edge_type, ctx=mx.gpu())
train_x = np.load('train_x.npy')
train_y = np.load('train_y.npy')
valid_x = np.load('valid_x.npy')
valid_y = np.load('valid_y.npy')
test_x = np.load('test_x.npy')
test_y = np.load('test_y.npy')

train_x = np.swapaxes(train_x, 1, 2)
train_x = np.concatenate([train_x[:, :, :, 0], train_x[:, :, :, 1]], axis=2)
train_y = np.swapaxes(train_y, 1, 2)
train_y = np.concatenate([train_y[:, :, :, 0], train_y[:, :, :, 1]], axis=2)

valid_x = np.swapaxes(valid_x, 1, 2)
valid_x = np.concatenate([valid_x[:, :, :, 0], valid_x[:, :, :, 1]], axis=2)
valid_y = np.swapaxes(valid_y, 1, 2)
valid_y = np.concatenate([valid_y[:, :, :, 0], valid_y[:, :, :, 1]], axis=2)

test_x = np.swapaxes(test_x, 1, 2)
test_x = np.concatenate([test_x[:, :, :, 0], test_x[:, :, :, 1]], axis=2)
test_y = np.swapaxes(test_y, 1, 2)
test_y = np.concatenate([test_y[:, :, :, 0], test_y[:, :, :, 1]], axis=2)

scaler_axis = (0, 1, 2)
scaler = StandardScaler(mean=train_x.mean(axis=scaler_axis),
                        std=train_x.std(axis=scaler_axis))
train_x = nd.array(train_x, ctx=mx.gpu())
train_y = nd.array(train_y, ctx=mx.gpu())
valid_x = nd.array(valid_x, ctx=mx.gpu())
valid_y = nd.array(valid_y, ctx=mx.gpu())
test_x = nd.array(test_x, ctx=mx.gpu())
test_y = nd.array(test_y, ctx=mx.gpu())

train_x = scaler.transform(train_x)
train_y = scaler.transform(train_y)
valid_x = scaler.transform(valid_x)
valid_y = scaler.transform(valid_y)
test_x = scaler.transform(test_x)
test_y = scaler.transform(test_y)

num_train = train_x.shape[0]
num_valid = valid_x.shape[0]
num_test = test_x.shape[0]

num_entra = tratim_matrix.shape[0]
seq_len = 4
pre_len = 4

# divide training set, valid set and test set
trainset = GraphTraffic(num_train, num_entra, tratim_matrix)
validset = GraphTraffic(num_valid, num_entra, tratim_matrix)
testset = GraphTraffic(num_test, num_entra, tratim_matrix)

for i in range(trainset.__len__()):
    trainset.graphs[i].ndata['h'] = train_x[i]
    trainset.graphs[i].edata['type'] = edge_type.reshape(-1, )
    trainset.labels[i] = train_y[i]
for i in range(validset.__len__()):
    validset.graphs[i].ndata['h'] = valid_x[i]
    validset.graphs[i].edata['type'] = edge_type.reshape(-1, )
    validset.labels[i] = valid_y[i]
for i in range(testset.__len__()):
    testset.graphs[i].ndata['h'] = test_x[i]
    testset.graphs[i].edata['type'] = edge_type.reshape(-1, )
    testset.labels[i] = test_y[i]

D:\anaconda\Anaconda\envs\jie_zeng\lib\site-packages\dgl\base.py:45: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  return warnings.warn(message, category=category, stacklevel=1)


number of nodes: 80
number of edges: 715


In [4]:
num_hidden = 224
inter_channel = 96
dropout = 0.05
n_hidden = num_hidden
in_feats = 2 * seq_len
num_classes = 2 * pre_len
lr = 1e-3
weight_decay = 0.0
n_bases = -1
n_layers = 1
n_epochs = 1
num_rels = 25
use_self_loop = True
batch_size = 32
test_batch_size = 8
# create model
model = SARGCN(in_feats,
               n_hidden,
               num_classes,
               num_rels,
               num_bases=n_bases,
               num_hidden_layers=n_layers,
               dropout=dropout,
               use_self_loop=use_self_loop,
               gpu_id=mx.gpu(),
               residual=False)
model.initialize(mx.init.Xavier(magnitude=math.sqrt(2.0)), ctx=mx.gpu())
trainer = gluon.Trainer(model.collect_params(),
                        'adam',
                        {'learning_rate': lr,
                         'wd': weight_decay})
loss = gluon.loss.L2Loss()
stopper = EarlyStopping(patience=15)
mx.random.seed(seed)
train_iter = gluon.data.DataLoader(trainset, batch_size, shuffle=False,
                                   batchify_fn=collate, last_batch='discard',
                                   num_workers=512, thread_pool=True)
mx.random.seed(seed)
valid_iter = gluon.data.DataLoader(validset, test_batch_size, shuffle=False,
                                   batchify_fn=collate, last_batch='discard',
                                   num_workers=512, thread_pool=True)
mx.random.seed(seed)
test_iter = gluon.data.DataLoader(testset, test_batch_size, shuffle=False,
                                  batchify_fn=collate, last_batch='discard',
                                  num_workers=512, thread_pool=True)

In [5]:
TrainLoss, ValidLoss, TestLoss = [], [], []
for epoch in range(n_epochs):
    train_l_sum, n = 0.0, 0
    start_time = time.time()
    for iter, (bg, label) in enumerate(train_iter):
        model.g = bg
        with mx.autograd.record():
            edge_type = bg.edata['type']
            pred = model(bg, bg.ndata['h'], edge_type, None)
            l = loss(pred, label)
        l.backward()
        trainer.step(batch_size)
        train_l_sum += l
        n += bg.__len__()
    train_l = train_l_sum
    train_rmse, train_loss = rmse_trainset(model, train_iter, batch_size, num_entra, scaler)
    valid_rmse, valid_loss = rmse_testset(model, valid_iter, test_batch_size, num_entra, scaler)
    test_rmse, test_loss = rmse_testset(model, test_iter, test_batch_size, num_entra, scaler)
    print(
        'epoch %d | running time %.2f s | train loss %.3f | valid loss %.3f | test loss %.3f | test rmse %.3f' %
        (epoch + 1, time.time() - start_time, train_loss, valid_loss, test_loss, test_rmse))
    TrainLoss.append(train_loss)
    ValidLoss.append(valid_loss)
    TestLoss.append(test_loss)
    if stopper.step(valid_loss, model):
        break

D:\anaconda\Anaconda\envs\jie_zeng\lib\site-packages\dgl\base.py:45: DGLWarning: DGLGraph.__len__ is deprecated.Please directly call DGLGraph.number_of_nodes.
  return warnings.warn(message, category=category, stacklevel=1)


epoch 1 | running time 15.52 s | train loss 8032.139 | valid loss 2960.285 | test loss 9520.906 | test rmse 137.992


In [19]:
model.load_parameters('pre_trained_model.param')
test = mx.nd.zeros(shape=(len(test_iter) * batch_size, num_entra, 8), ctx=mx.gpu())
pred = mx.nd.zeros(shape=(len(test_iter) * batch_size, num_entra, 8), ctx=mx.gpu())
count = 0
for iter, (bg, label) in enumerate(test_iter):
    y = label
    edge_type = bg.edata['type']
    _ = model(bg, bg.ndata['h'], edge_type, None)
    test[count: count + len(label), :] = scaler.inverse_transform(y)
    pred[count: count + len(label), :] = scaler.inverse_transform(_).reshape(-1, num_entra, 8)
    count += len(label)

test = test.asnumpy()
pred = pred.asnumpy()
for i in range(4):
    test_ = test[:, :, [i, i+4]]
    pred_ = pred[:, :, [i, i+4]]
    rmse = masked_rmse_np(test_, pred_)
    mae = masked_mae_np(test_, pred_)
    mape = masked_mape_np(test_, pred_)
    print('time: %d min, rmse: %.3f, mae: %.3f, mape: %.5f' % ((i+1)*15, rmse, mae, mape))

time: 15 min, rmse: 36.413, mae: 22.599, mape: 0.14460
time: 30 min, rmse: 38.358, mae: 23.633, mape: 0.15164
time: 45 min, rmse: 39.488, mae: 24.489, mape: 0.16504
time: 60 min, rmse: 42.436, mae: 25.740, mape: 0.19270
